<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#PAPi---Pathway-Activity-Profiling" data-toc-modified-id="PAPi---Pathway-Activity-Profiling-1">PAPi - Pathway Activity Profiling</a></span><ul class="toc-item"><li><span><a href="#Make-Compound-and-Pathway-Databases" data-toc-modified-id="Make-Compound-and-Pathway-Databases-1.1">Make Compound and Pathway Databases</a></span></li><li><span><a href="#Perform-PAPi" data-toc-modified-id="Perform-PAPi-1.2">Perform PAPi</a></span></li><li><span><a href="#Post-Hoc-PAPi" data-toc-modified-id="Post-Hoc-PAPi-1.3">Post Hoc PAPi</a></span></li><li><span><a href="#Papi-All-Organs" data-toc-modified-id="Papi-All-Organs-1.4">Papi All Organs</a></span></li></ul></li></ul></div>

# PAPi - Pathway Activity Profiling

## Make Compound and Pathway Databases

In [1]:
library(PAPi)
library(tidyverse)

ERROR: Error in library(PAPi): there is no package called ‘PAPi’


In [8]:
compounds <- read_csv('../PAPi/all_kegg_compounds.csv')

Parsed with column specification:
cols(
  compound = col_character(),
  CmpdID = col_character()
)


In [9]:
# taken from https://bioconductor.org/packages/release/bioc/html/PAPi.html
getPathways <- function(x){
    TotalReport <- keggGet(x)
    pathways <- TotalReport[[1]]$PATHWAY
    pathways <- data.frame(pathways)
    pathways <- row.names(pathways)
    pathways <- paste(pathways, collapse = ";")
    return(pathways)
}

listOfComps <- keggList(compounds[['CmpdID']])
listOfComps <- data.frame(listOfComps)
listOfComps[2] <- row.names(listOfComps)
row.names(listOfComps) <- 1:nrow(listOfComps)
listOfComps[3] <- apply(listOfComps[2], 1, function(x) getPathways(x))

In [10]:
numbcomp <- function(x){
    compounds <- keggGet(x)
    compounds <- data.frame(compounds[[1]]$COMPOUND)
    compounds <- nrow(compounds)
    if (compounds == 0){
        x <- gsub("path:mmu", "ko", x, fixed = TRUE)
        compounds <- try(keggGet(x), silent = TRUE)
        if("try-error" %in% class(compounds)){
            compounds <- 0
        } else {
            compounds <- data.frame(compounds[[1]]$COMPOUND)
            compounds <- nrow(compounds)
        }
    }
    return(compounds)
}

pathname <- data.frame(keggList("pathway", "mmu"))
pathname[2] <- row.names(pathname)
row.names(pathname) <- 1:nrow(pathname)
pathname[3] <- apply(pathname[2], 1, function(x) numbcomp(x))

In [11]:
pathname[[1]] <- gsub("- Mus musculus \\(mouse)", "", pathname[[1]])
pathname[[2]] <- gsub("path:mmu", "map", pathname[[2]])

In [12]:
head(pathname)

keggList..pathway....mmu..,V2,V3
<chr>,<chr>,<int>
Glycolysis / Gluconeogenesis,map00010,31
Citrate cycle (TCA cycle),map00020,20
Pentose phosphate pathway,map00030,35
Pentose and glucuronate interconversions,map00040,55
Fructose and mannose metabolism,map00051,54
Galactose metabolism,map00052,46


In [100]:
# write databases to files
#write_csv(listOfComps, '../PAPi/COMPbase.csv')#, row.names = FALSE)
#write_csv(pathname, '../PAPi/PATHbase.csv')#, row.names = FALSE)

## Perform PAPi

In [13]:
getcomp <- function(x){
    comp <- which(listOfComps[[2]] == x)
    if (length(comp) > 0){
        comp <- listOfComps[comp[1], 3]
        #comp <- gsub("map", "path:mmu", comp)
        comp <- data.frame(unlist(strsplit(comp, ";")))
        pathwaysFound <- which(pathname[,2] %in% comp[,1])
        if (length(pathwaysFound) > 0){
            pathwaysFound <- pathname[pathwaysFound,]
            row.names(pathwaysFound) <- pathwaysFound[,2]
            pathwaysFound <- pathwaysFound[1]
            names(pathwaysFound)[1] <- "value"
        } else {
            pathwaysFound <- 0
        }
    } else {
        pathwaysFound <- 0
    }
    return(pathwaysFound)
}
##
## Function to go to KEGG and collect compounds playing part in each pathway
numbcomp <- function(x){
    compounds <- which(pathname[2] == x)
    if (length(compounds) > 0){
        compounds <- as.numeric(pathname[compounds[1], 3])
    } else {
        compounds <- 0
    }
    return(compounds)
}

In [14]:
papi2 <- function(omics.data.frame) {
    replicates <- as.character(omics.data.frame[1, ])
    omics.data.frame <- omics.data.frame[-1, ]
    rep <- 1
    reps <- factor(replicates[-1])

    ## Apply PAPi for each column of the input data
    ## Starts the progress bar
    pb <- txtProgressBar(min = 0, max = ncol(omics.data.frame), style = 3, width=50)
    papi.frame <- numeric()
    for(j in 2:ncol(omics.data.frame)){
        Sys.sleep(0.01)
        setTxtProgressBar(pb, j)
        ## Prepare the list of compunds that will be searched in KEGG
        data.df <- omics.data.frame[,c(1,j)]
        data.df <- subset(data.df, !is.na(data.df[2]))
        complist <- as.character(data.df[,1])
        complist <- gsub("C", "cpd:C", complist, ignore.case = TRUE)
        ## Give to pathways the abundance of the related metabolite
        getpath.final <- numeric()
        for (i in 1:length(complist)){
            getpath <- getcomp(complist[i])
            if (sum(getpath != 0) != 0){
                if(length(getpath.final) == 0){
                    getpath$rate <- as.numeric(data.df[i,2])
                    getpath.final <- getpath
                } else {
                    getpath$rate <- as.numeric(data.df[i,2])
                    getpath.final <- rbind(getpath, getpath.final)
                }
            } else {
            }
        }

        ## Calculate rates ###
        getpath.final$rate <- as.numeric(as.character(getpath.final$rate))
        res.arr <- with(getpath.final, tapply(rate, value, sum))
        res.df <- data.frame(pathwayname = names(res.arr), rate = res.arr)
        rownames(res.df) <- NULL
        pathwayfreq <- data.frame(table(getpath.final$value))
        names(pathwayfreq)[1] <- "pathwayname"
        names(pathname)[c(1,2)] <- c("pathwayname", "idpathway")
        freqname <- merge(pathwayfreq, pathname)
        freqname$idpathway <- gsub("path:map", "ko", freqname$idpathway)
        total <- nrow(pathwayfreq)
        ## Starts the final calculations - Normalization
        selecrow.final <- numeric()
        for (i in 1:nrow(res.df)){
            selecrow <- freqname[i,]
            numberCompounds <- try(numbcomp(selecrow$idpathway), silent = TRUE)
            if ("try-error" %in% class(numberCompounds)){
                selecrow$percentage <- 1
            } else {
                if (numberCompounds != 0){
                    selecrow$percentage <- (selecrow$Freq/numberCompounds)
                } else {
                    selecrow$percentage <- 1
                }
                if(i == 1){
                    selecrow.final <- selecrow
                } else {
                    selecrow.final <- rbind(selecrow.final, selecrow)
                }
            }
        }
        selecrow.final <- selecrow.final[,-c(2,3,4)]
        final2.df <- merge(selecrow.final, res.df)
        final2.df[2] <- final2.df$rate/final2.df$percentage
        names(final2.df)[2] <- names(data.df)[2]
        final2.df[3] <- NULL
        if(j == 2){
            papi.frame <- final2.df
        } else {
            papi.frame <- merge(papi.frame, final2.df, all = TRUE)
        }
    }
    close(pb)
    ## Finish PAPi algorithm
    papi.frame <- papi.frame[order(papi.frame[,2], decreasing = T),]
    papi.frame[1] <- as.character(papi.frame[,1])
    if (rep == 1) {
        papi.frame <- rbind(c(replicates), papi.frame)
    }
    return (papi.frame)
}
    

In [15]:
for (i in c('Heart','Liver','Lung','Kidney','Plasma')) {
    organ <- read_csv(paste('../Data/Normalized/', i, '_normalized.csv', sep = ''), col_names = F)
    organ <- as.data.frame(t(organ), stringsAsFactors = FALSE)
    colnames(organ) <- c('Names', organ[1,2:ncol(organ)])
    organ <- organ[2:nrow(organ),]
    organ[1,1] <- 'Replicates'
    rownames(organ) <- NULL
    all_papi <- papi2(organ)
    all_papiH <- papiHtest(all_papi, save = F, signif.level = 0.05, StatTest = 'ANOVA', log.transform = F, adjust.pValue = F)
    write_csv(all_papiH[,-ncol(all_papiH)], paste('../PAPi/', i, '/', i,'_all_papi.csv', sep=''))
    }

Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


  |==================================================| 100%


Data frame loaded...
ANOVA is in progress.. 

Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


  |==================================================| 100%


Data frame loaded...
ANOVA is in progress.. 

Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


...

Data frame loaded...
ANOVA is in progress.. 

Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


  |==================================================| 100%


Data frame loaded...
ANOVA is in progress.. 

Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


  |==================================================| 100%


Data frame loaded...
ANOVA is in progress.. 



## Post Hoc PAPi

In [72]:
library(reshape2)
library(broom)
library(rowr)
library(plyr)
library(FactoMineR)

------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: ‘plyr’

The following object is masked from ‘package:rowr’:

    count

The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following object is masked from ‘package:purrr’:

    compact



In [73]:
# change labels columns into factor
group_order = c('Normal', '4 Hr Sham', '4 Hr AKI', '24 Hr Sham', '24 Hr AKI', '7 Day Sham', '7 Day AKI') # order
factor_labels <- function(data_frame) {
    data_frame$Replicates = factor(data_frame$Replicates, levels = group_order, ordered = T)
    return (data_frame[order(data_frame$Replicates),])
}

In [113]:
organ_name <- 'Plasma'
organ <- t(read_csv(paste('../PAPi/', organ_name, '/', organ_name, '_all_papi.csv', sep = ''), col_names = F)) %>% as_tibble()

Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


In [114]:
names(organ) <- organ[1,]
organ <- organ[-1,]

In [115]:
organ[-c(1,2)] <- apply(organ[-c(1,2)], 2, function(x) {as.numeric(x)})
data_frame <- factor_labels(organ)

In [116]:
multiT <- function(feature_column, labels) {
    met <- colnames(feature_column)
    feature_column <- cbind(labels,feature_column)
    colnames(feature_column) = c('Label', met)
    
    test <- feature_column %>% group_by(Label) %>% mutate(grouped_id = row_number(Label)) %>% # manipulate column into multi column
        spread(Label, met) %>% select(-grouped_id) %>% select(group_order)
    
    v1 = c(1,1,1,1,1,1,2,4,6) # which groups do you want to compare normal == 1, 4 hr sham == 2, etc. 
    v2 = c(2,3,4,5,6,7,3,5,7)
    combos = rbind(v1,v2)
    adply(combos, 2, function(x) { # do dem t tests
    p <- t.test(as.matrix(test[!is.na(test[,x[1]]), x[1]]), as.matrix(test[!is.na(test[,x[2]]), x[2]]))

    out <- data.frame("var1" = colnames(test)[x[1]] , "var2" = colnames(test[x[2]]), 
                  "t.value" = sprintf("%.3f", p$statistic) ,  "df"= p$parameter ,  "p.value" = p$p.value)
  return(out)
})
}

In [117]:
# Benjamini, Hochberg, and Yekutieli method for mult comparisons 
# https://stats.stackexchange.com/questions/63441/what-are-the-practical-differences-between-the-benjamini-hochberg-1995-and-t
p_values = list()
for (met in colnames(data_frame[,-c(1,2)])) {
    val = multiT(data_frame[,met], data_frame$Replicates)
    val$p.value = p.adjust(val$p.value, method = 'BY')
    p_values[[met]] <- val
}

In [118]:
get_sig_mets <- function(p_values, SHAM, AKI, comparison_cat, i) {
    # Significant to AKI
    increased = NA
    decreased = NA
    if(comparison_cat == 'Sig2AKI') {
    # Increased AKI
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == AKI), ]$p.value <= 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 & 
                p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value > 0.05 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) < 0) {increased = names(p_values[i])}
                    # this is a factor, need to change to int, using the t.test value to infer direction inc vs dec
    # decreased AKI
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == AKI), ]$p.value <= 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 & 
                p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value > 0.05 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) > 0) {decreased = names(p_values[i])}
    }
    
    # more in AKI
    if(comparison_cat == 'MoreInAKI') {
    # Increased More in 4 hr AKI
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 & 
                as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$t.value)) < 0 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) < 0) {increased = names(p_values[i])}
    # decreased more in 4 hr AKI
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 & 
                as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$t.value)) > 0 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) > 0) {decreased = names(p_values[i])}
    }
    
     # more in AKI
    if(comparison_cat == 'OppInAKI') {
    # increased opposite
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 &
                p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == AKI), ]$p.value <= 0.05 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$t.value)) > 0 &
                        as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) < 0) {increased = names(p_values[i])}
    # decreased opposite
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 &
                p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == AKI), ]$p.value <= 0.05 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$t.value)) < 0 &
                        as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) > 0) {decreased = names(p_values[i])}
    }
    
    if(comparison_cat == 'NoEfAKI') {
    # No effect in AKI
    # increase noef AKI
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == AKI), ]$p.value > 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 & 
                p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) > 0) {increased = names(p_values[i])}
    # decrease  no effect AKI
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == AKI), ]$p.value > 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 & 
                p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) < 0) {decreased = names(p_values[i])}
    }
    
    if(comparison_cat == 'Sig2Sham') {
    # increase sham
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 & 
            as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$t.value)) < 0) {increased = names(p_values[i])}
    #decrease sham
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 & 
            as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$t.value)) > 0) {decreased = names(p_values[i])}
    }
    return (list(increased, decreased))
}

In [119]:
get_list <- function(p_values, SHAM, AKI, cat) {
    inc = c()
    dec = c()
    for (i in 1:length(p_values)) {
        vals <- get_sig_mets(p_values, SHAM, AKI, cat, i)
        inc <- append(inc, vals[1])
        dec <- append(dec, vals[2])
        }
    inc = inc[!is.na(inc)]
    dec = dec[!is.na(dec)]
    return (list(inc,dec))
}

In [120]:
#sig2AKI
Sig4AKI = get_list(p_values, '4 Hr Sham', '4 Hr AKI', 'Sig2AKI')
ISig4AKI = Sig4AKI[1]
DSig4AKI = Sig4AKI[2]

Sig24AKI = get_list(p_values, '24 Hr Sham', '24 Hr AKI', 'Sig2AKI')
ISig24AKI = Sig24AKI[1]
DSig24AKI = Sig24AKI[2]

Sig7AKI = get_list(p_values, '7 Day Sham', '7 Day AKI', 'Sig2AKI')
ISig7AKI = Sig7AKI[1]
DSig7AKI = Sig7AKI[2]

#more in AKI
More4AKI = get_list(p_values, '4 Hr Sham', '4 Hr AKI', 'MoreInAKI')
IMore4AKI = More4AKI[1]
DMore4AKI = More4AKI[2]

More24AKI = get_list(p_values, '24 Hr Sham', '24 Hr AKI', 'MoreInAKI')
IMore24AKI = More24AKI[1]
DMore24AKI = More24AKI[2]

More7AKI = get_list(p_values, '7 Day Sham', '7 Day AKI', 'MoreInAKI')
IMore7AKI = More7AKI[1]
DMore7AKI = More7AKI[2]

# opposite effect in AKI
Opp4AKI = get_list(p_values, '4 Hr Sham', '4 Hr AKI', 'OppInAKI')
IOpp4AKI = Opp4AKI[1]
DOpp4AKI = Opp4AKI[2]

Opp24AKI = get_list(p_values, '24 Hr Sham', '24 Hr AKI', 'OppInAKI')
IOpp24AKI = Opp24AKI[1]
DOpp24AKI = Opp24AKI[2]

Opp7AKI = get_list(p_values, '7 Day Sham', '7 Day AKI', 'OppInAKI')
IOpp7AKI = Opp7AKI[1]
DOpp7AKI = Opp7AKI[2]

# no effect in AKI, but effect in sham
NoEf4 = get_list(p_values, '4 Hr Sham', '4 Hr AKI', 'NoEfAKI')
INoef4AKI = NoEf4[1]
DNoef4AKI = NoEf4[2]

NoEf24 = get_list(p_values, '24 Hr Sham', '24 Hr AKI', 'NoEfAKI')
INoef24AKI = NoEf24[1]
DNoef24AKI = NoEf24[2]

NoEf7 = get_list(p_values, '7 Day Sham', '7 Day AKI', 'NoEfAKI')
INoef7AKI = NoEf7[1]
DNoef7AKI = NoEf7[2]

# effect in sham
Sig4Sham = get_list(p_values, '4 Hr Sham', '4 Hr AKI', 'Sig2Sham')
ISig4Sham = Sig4Sham[1]
DSig4Sham = Sig4Sham[2]

Sig24Sham = get_list(p_values, '24 Hr Sham', '24 Hr AKI', 'Sig2Sham')
ISig24Sham = Sig24Sham[1]
DSig24Sham = Sig24Sham[2]

Sig7Sham = get_list(p_values, '7 Day Sham', '7 Day AKI', 'Sig2Sham')
ISig7Sham = Sig7Sham[1]
DSig7Sham = Sig7Sham[2]

In [121]:
Sig2AKI = as.data.frame(cbind.fill(ISig4AKI[[1]], ISig24AKI[[1]], ISig7AKI[[1]], DSig4AKI[[1]], DSig24AKI[[1]], DSig7AKI[[1]], fill = "")) 
colnames(Sig2AKI) = c("Increase 4 Hr AKI", "Increase 24 Hour AKI", "Increase 7 Day AKI", "Decrease 4 Hr AKI", "Decrease 24 Hour AKI", "Decrease 7 Day AKI")

MoreInAKI = as.data.frame(cbind.fill(IMore4AKI[[1]], IMore24AKI[[1]], IMore7AKI[[1]], DMore4AKI[[1]], DMore24AKI[[1]], DMore7AKI[[1]], fill = "")) 
if (dim(MoreInAKI)[1] != 0) {
    colnames(MoreInAKI) = c("More Increase 4 Hr AKI", "More Increase 24 Hour AKI", "More Increase 7 Day AKI", "More Decrease 4 Hr AKI", "More Decrease 24 Hour AKI", "More Decrease 7 Day AKI")
}

NoefAKI = as.data.frame(cbind.fill(INoef4AKI[[1]], INoef24AKI[[1]], INoef7AKI[[1]], DNoef4AKI[[1]], DNoef24AKI[[1]], DNoef7AKI[[1]], fill = "")) 
if (dim(NoefAKI)[1] != 0) {
    colnames(NoefAKI) = c("NoEf Increase 4 Hr AKI", "NoEf Increase 24 Hour AKI", "NoEf Increase 7 Day AKI", "NoEf Decrease 4 Hr AKI", "NoEf Decrease 24 Hour AKI", "NoEf Decrease 7 Day AKI")
}

OppAKI = as.data.frame(cbind.fill(IOpp4AKI[[1]], IOpp24AKI[[1]], IOpp7AKI[[1]], DOpp4AKI[[1]], DOpp24AKI[[1]], DOpp7AKI[[1]], fill = "")) 
if (dim(OppAKI)[1] != 0) {
    colnames(OppAKI) = c("Opp Increase 4 Hr AKI", "Opp Increase 24 Hour AKI", "Opp Increase 7 Day AKI", "Opp Decrease 4 Hr AKI", "Opp Decrease 24 Hour AKI", "Opp Decrease 7 Day AKI")
}

Sig2Sham = as.data.frame(cbind.fill(ISig4Sham[[1]], ISig24Sham[[1]], ISig7Sham[[1]], DSig4Sham[[1]], DSig24Sham[[1]], DSig7Sham[[1]], fill = "")) 
if (dim(Sig2Sham)[1] != 0) {
    colnames(Sig2Sham) = c("Increase 4 Hour Effect of Sham", "Increase 24 Hour Effect of Sham", "Increase 7 Day Effect of Sham", "Decrease 4 Hr Effect of Sham", "Decrease 24 Hour Effect of Sham", "Decrease 7 Day Effect of Sham")
}
Final = cbind.fill(Sig2AKI, MoreInAKI, NoefAKI, OppAKI, Sig2Sham, fill = "")
write.csv(Final, paste('../PAPi/',organ_name,'/',organ_name,'_PAPi_posthoc_table.csv', sep= ''))

Warning message in FUN(X[[i]], ...):
“'x' is NULL so the result will be NULL”Warning message in FUN(X[[i]], ...):
“'x' is NULL so the result will be NULL”Warning message in FUN(X[[i]], ...):
“'x' is NULL so the result will be NULL”Warning message in FUN(X[[i]], ...):
“'x' is NULL so the result will be NULL”Warning message in FUN(X[[i]], ...):
“'x' is NULL so the result will be NULL”Warning message in FUN(X[[i]], ...):
“'x' is NULL so the result will be NULL”Warning message in FUN(X[[i]], ...):
“'x' is NULL so the result will be NULL”Warning message in FUN(X[[i]], ...):
“'x' is NULL so the result will be NULL”Warning message in FUN(X[[i]], ...):
“'x' is NULL so the result will be NULL”Warning message in FUN(X[[i]], ...):
“'x' is NULL so the result will be NULL”Warning message in FUN(X[[i]], ...):
“'x' is NULL so the result will be NULL”

## Papi All Organs

In [144]:
papipapi <- function(organ_name) {
    organ <- read_csv(paste('../Data/Normalized/', organ_name, '_normalized.csv', sep = ''), col_names = F)
    organ <- as.data.frame(t(organ), stringsAsFactors = FALSE)
    colnames(organ) <- c('Names', organ[1,2:ncol(organ)])
    organ <- organ[2:nrow(organ),]
    organ[1,1] <- 'Replicates'
    rownames(organ) <- NULL
    all_papi <- papi2(organ)
    all_papiH <- papiHtest(all_papi, save = F, signif.level = 0.05, StatTest = 'ANOVA', log.transform = F, adjust.pValue = F)
    all_papiH <- rbind(organ_name, all_papiH)
    all_papiH[1,1] <- 'Organ'
    return (all_papiH)
}

In [145]:
papi_dfs <- list(papipapi('Heart'), papipapi('Lung'), papipapi('Liver'), papipapi('Plasma'), papipapi('Kidney'))
names(papi_dfs) <- c('Heart', 'Lung', 'Liver','Plasma','Kidney')

Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


  |==================================================| 100%


Data frame loaded...
ANOVA is in progress.. 

Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


  |==================================================| 100%


Data frame loaded...
ANOVA is in progress.. 

Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


...

Data frame loaded...
ANOVA is in progress.. 

Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


  |==================================================| 100%


Data frame loaded...
ANOVA is in progress.. 

Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


  |==================================================| 100%


Data frame loaded...
ANOVA is in progress.. 



In [147]:
papi_dfs$Heart

,pathwayname,523,526,529,532,535,547,550,553,556,⋯,504,508,510,496,498,500,501,503,505,pvalues
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Organ,Heart,Heart,Heart,Heart,Heart,Heart,Heart,Heart,Heart,⋯,Heart,Heart,Heart,Heart,Heart,Heart,Heart,Heart,Heart,Heart
18,Replicates,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,⋯,7 Day Sham,7 Day Sham,7 Day Sham,7 Day AKI,7 Day AKI,7 Day AKI,7 Day AKI,7 Day AKI,7 Day AKI,Not-adjusted
30,2-Oxocarboxylic acid metabolism,1.32993606159162,1.30515683845536,1.23818378447985,-0.350998480468472,0.501851123309744,0.644750618273754,1.1108382567653,0.127862392486999,0.0399074782583104,⋯,1.81702626691509,-0.231352466983604,1.46072507762668,-1.70059361529267,-0.115551598444644,0.279700759450094,-0.752199183907122,-0.216981224129605,0.889440710342524,3.49953425296086e-07
60,AGE-RAGE signaling pathway in diabetic complications,-1.13866545086964,-0.824562683961176,-0.39564916124023,0.137184781224677,-0.229528350707148,-1.53729408224717,-1.79994252643064,-1.2530263183,-1.63356545036344,⋯,0.701876326107213,-0.592071947485668,0.0785996327298735,5.56287283836493,-0.0661305987569987,4.72701059659201,5.64677607249924,2.71720543432847,3.6117619747311,2.29636162675713e-18
71,"Alanine, aspartate and glutamate metabolism",2.94731101572682,3.39180130499281,3.19469863643811,-0.32332420222697,1.7959327572742,0.48848825955871,0.912164405791477,-1.11784034495683,-0.649839892467565,⋯,2.10461831622172,-2.27037656181406,2.98144950447841,-5.75145437718174,0.383527154966266,1.28661827780338,-0.466685607655781,-0.120787586557643,2.47011609010823,0.0181244751218178
82,Alcoholism,1.39002044125736,1.10379608790921,1.83086147937834,0.472666542908598,0.597217081281076,1.58353045619793,2.414688014634,0.692860901843615,-0.051503049310933,⋯,2.30902760777315,-0.00997280303392833,3.66384100434444,-1.63783185972315,0.129673950230558,0.205894197644704,-0.556449575646187,2.14328552065566,1.14120911015164,7.8382890648557e-18
93,Aldosterone synthesis and secretion,2.79749206583368,3.64563467802336,2.69963547792347,-1.62263889306566,2.16331528457591,1.91643936101184,-1.10958691786804,-1.60111631198803,-7.06529294381056,⋯,6.31070764423013,5.28080717562115,5.10266688451151,0.103047274560583,6.37432800005227,6.23552155793346,4.06245180424999,7.67592000210342,6.10700946005993,4.14312492845128e-07
104,Amino sugar and nucleotide sugar metabolism,-13.6639854104357,-9.89475220753411,-4.74778993488276,1.64621737469613,-2.75434020848578,-18.4475289869661,-21.5993103171677,-15.0363158196,-19.6027854043612,⋯,8.42251591328656,-7.10486336982802,0.943195592758482,66.7544740603792,-0.793567185083984,56.7241271591041,67.7613128699909,32.6064652119416,43.3411436967732,2.29636162675713e-18
115,Aminoacyl-tRNA biosynthesis,0.713701002960293,9.06039882332027,2.52616811641889,-1.91664063003789,-3.95208559202508,4.36186830467078,1.58344223888825,2.62062044380257,2.12061264627856,⋯,5.26150848090058,-3.49894392862246,2.75704944412987,-4.53013170604706,0.285103875387966,-2.83225586503738,-4.67599284261952,-1.05533802674185,-2.23263050398824,4.77855732240553e-05


In [148]:
#merge(x, y, by, by.x, by.y, sort = TRUE) 
#for (i in c('Heart','Liver','Lung','Kidney','Plasma')) { 
f <- papi_dfs[['Heart']][, papi_dfs[['Heart']][2,] %in% c('Replicates','4 Hr AKI')]
tf <- papi_dfs[['Heart']][, papi_dfs[['Heart']][2,] %in% c('Replicates','24 Hr AKI')]
s <- papi_dfs[['Heart']][, papi_dfs[['Heart']][2,] %in% c('Replicates','7 Day AKI')]

for (i in c('Liver','Lung','Kidney','Plasma')) { 
    f <- join(f, papi_dfs[[i]][, papi_dfs[[i]][2,] %in% c('Replicates','4 Hr AKI')], by = 'pathwayname', type = 'inner')
    tf <- join(tf, papi_dfs[[i]][, papi_dfs[[i]][2,] %in% c('Replicates','24 Hr AKI')], by = 'pathwayname', type = 'inner')
    s <- join(s, papi_dfs[[i]][, papi_dfs[[i]][2,] %in% c('Replicates','7 Day AKI')], by = 'pathwayname', type = 'inner')
}

In [150]:
write_csv(f[-2,], '../PAPi/4_all_organs.csv')
write_csv(tf[-2,], '../PAPi/24_all_organs.csv')
write_csv(s[-2,], '../PAPi/7d_all_organs.csv')

In [ ]:
f_hour <- cbind(   organ[, organ[1, ] %in% c('Replicates', 'Normal', '4 Hr Sham', '4 Hr AKI')]
tf_hour <- organ[, organ[1, ] %in% c('Replicates', 'Normal', '24 Hr Sham', '24 Hr AKI')]
s_day <- organ[, organ[1, ] %in% c('Replicates', 'Normal', '7 Day Sham', '7 Day AKI')]

In [50]:
# f_hour <- organ[, organ[1, ] %in% c('Replicates', 'Normal', '4 Hr Sham', '4 Hr AKI')]
# tf_hour <- organ[, organ[1, ] %in% c('Replicates', 'Normal', '24 Hr Sham', '24 Hr AKI')]
# s_day <- organ[, organ[1, ] %in% c('Replicates', 'Normal', '7 Day Sham', '7 Day AKI')]

In [54]:
# f_hr_papi <- papi2(f_hour)
# tf_hour_papi <- papi2(tf_hour)
# s_day_papi <- papi2(s_day)

  |==================================================| 100%
  |==================================================| 100%
  |==================================================| 100%


In [87]:
# f_hour_papiH <- papiHtest(f_hr_papi, save = F, signif.level = 0.05, StatTest = 'ANOVA', log.transform = F, adjust.pValue = F)
# tf_hour_papiH <- papiHtest(tf_hour_papi, save = F, signif.level = 0.05, StatTest = 'ANOVA', log.transform = F, adjust.pValue = F)
# s_day_papiH <- papiHtest(s_day_papi, save = F, signif.level = 0.05, StatTest = 'ANOVA', log.transform = F, adjust.pValue = F)

Data frame loaded...
ANOVA is in progress.. 

Data frame loaded...
ANOVA is in progress.. 

Data frame loaded...
ANOVA is in progress.. 



In [98]:
#f_outliers = which(f_hour_papiH[,1] == 'Primary bile acid biosynthesis ')
#tf_outliers = which(tf_hour_papiH[,1] == 'Primary bile acid biosynthesis ')
#s_outliers = which(s_day_papiH[,1] == 'Primary bile acid biosynthesis ')

In [99]:
# papiLine(f_hour_papiH[,-ncol(f_hour_papiH)], setRef.cond= T, Ref.cond = 'Normal', relative = T, graphWidth = 5000, graphHeight = 3000, output = '4hr', save = T)
# papiLine(tf_hour_papiH[-tf_outliers, -ncol(tf_hour_papiH)], setRef.cond= T, Ref.cond = 'Normal', relative = T, graphWidth = 5000, graphHeight = 3000, output = '24hr', save = T)
# papiLine(s_day_papiH[,-ncol(s_day_papiH)], setRef.cond= T, Ref.cond = 'Normal', relative = T, graphWidth = 5000, graphHeight = 3000,  output = '7day', save = T)

[1] "Data frame loaded..."
[1] "No folder was defined to save the results."
[1] "Please, point to the folder where the results should be saved."
[1] "The file 4hr.png has been saved in /Users/benfox/Documents/Faubel/MS Project/Notebooks"
[1] "Data frame loaded..."
[1] "No folder was defined to save the results."
[1] "Please, point to the folder where the results should be saved."
[1] "The file 24hr.png has been saved in /Users/benfox/Documents/Faubel/MS Project/Notebooks"
[1] "Data frame loaded..."
[1] "No folder was defined to save the results."
[1] "Please, point to the folder where the results should be saved."
[1] "The file 7day.png has been saved in /Users/benfox/Documents/Faubel/MS Project/Notebooks"


In [102]:
all_papi <- papi2(organ)

  |==================================================| 100%


In [122]:
all_papiH <- papiHtest(all_papi, save = F, signif.level = 0.05, StatTest = 'ANOVA', log.transform = F, adjust.pValue = F)

Data frame loaded...
ANOVA is in progress.. 



In [143]:
all_rbind('Heart',all_papiH)
all_papiH[1,1] <- 'Organ'

,pathwayname,523,526,529,532,535,547,550,553,556,⋯,504,508,510,496,498,500,501,503,505,pvalues
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Heart,Heart,Heart,Heart,Heart,Heart,Heart,Heart,Heart,Heart,⋯,Heart,Heart,Heart,Heart,Heart,Heart,Heart,Heart,Heart,Heart
120,Replicates,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,⋯,7 Day Sham,7 Day Sham,7 Day Sham,7 Day AKI,7 Day AKI,7 Day AKI,7 Day AKI,7 Day AKI,7 Day AKI,Not-adjusted
26,2-Oxocarboxylic acid metabolism,0.129548630974285,0.826408185832601,0.650201248784231,-0.151037637631177,-0.224480985419002,0.870703963823419,-0.494042933566706,2.33920431718354,0.823992250581995,⋯,0.916348855214587,0.479107459293659,1.71930427923763,1.30084225976344,1.05000438087443,0.258827364245658,0.00690074422691348,1.85162788549809,1.15240422327613,4.46979543288495e-14
34,ABC transporters,-2.10096043093755,14.3654078174045,2.62135231884978,3.55138622633051,-1.40539314337642,5.39878916478217,-1.28303031255646,26.7300390049206,9.19902179917466,⋯,5.1370405044848,-6.21793009894079,-1.07799069574004,2.15973160316525,-4.70896702548271,-9.66615377340215,2.93604000387072,2.7294406301566,-6.0947952562836,1.96774462510207e-12
45,African trypanosomiasis,-1.44232375172434,1.46568876160814,-2.20482991716669,-2.25422771863382,-2.00700257650603,-2.010974204005,-2.01050124997206,-2.62070070263877,-1.38614648615266,⋯,2.64395118144464,1.16076231486379,1.98460580442241,1.5954282890418,3.90042098813618,2.2187420064298,1.54843632465972,2.67825803042329,2.24301027487337,1.45101240305051e-08
56,AGE-RAGE signaling pathway in diabetic complications,0.228761004631387,0.44977047055923,2.06716762366907,3.89835275108483,5.64121654080695,0.701929625436177,2.39589556352493,3.25539048099791,4.62222682640001,⋯,-1.59724034299466,-0.167341607810408,-0.217199552596313,-1.55561822553422,0.109720302226513,-1.58460363669878,-0.934586218812969,-2.90581114820725,-0.755699709899711,1.3028693142646e-08
67,"Alanine, aspartate and glutamate metabolism",1.18382568763116,-1.36815657207007,1.78876660599119,-0.0479387656903997,0.0884973525942195,-2.89553818708212,-3.79165548068476,-0.679868561619763,-4.6236545684804,⋯,-0.175587399773168,0.891229242896992,1.94743187428805,4.34348623812643,2.95004274976995,-0.0943595387110752,1.48459583407337,5.76521709588078,2.86717760894373,0.000967503708500095
78,Alcoholism,0.273216494471811,-0.307550836013412,0.636581489378795,0.421140974558961,2.37574726759814,0.209124675232317,0.278198423930935,1.52043926170142,1.81230032686833,⋯,-0.28480692242012,-1.01481910010379,-0.321690375233256,-1.18988944629494,-1.75069315907414,-1.44221199113621,1.97803802311128,1.65990509824666,-1.57289153958851,0.000369565300434372
89,Aldosterone synthesis and secretion,2.98827692853703,1.18140469671347,-0.0989896000949011,3.60386466102643,6.37122421347964,-3.62260352458354,-3.37992521100153,-3.40483578300404,3.63239604990286,⋯,3.008851212926,1.12258281764537,-2.73668718336464,-0.705583798520781,-4.31633830428907,-2.62691128875734,0.0987472753820485,-0.307625702726914,-1.62165381094017,0.0399548161243632


In [107]:
# write papiANOVAs to files
write_csv(all_papiH[,-ncol(all_papiH)], '../PAPi/organ_all_papi.csv')